In [12]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re

In [13]:
df = pd.read_csv('ACS_17_5YR_S1901_with_ann.csv')

In [14]:
df.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,...,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20
0,Id,Id2,Geography,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...,Households; Estimate; PERCENT ALLOCATED - Nonf...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Nonfam...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...
1,1400000US13007960100,13007960100,"Census Tract 9601, Baker County, Georgia",804,99,528,69,345,78,276,...,(X),(X),(X),(X),(X),(X),(X),(X),59.8,(X)
2,1400000US13007960200,13007960200,"Census Tract 9602, Baker County, Georgia",517,83,311,66,253,61,206,...,(X),(X),(X),(X),(X),(X),(X),(X),52.9,(X)
3,1400000US13095000100,13095000100,"Census Tract 1, Dougherty County, Georgia",1905,220,911,173,244,117,994,...,(X),(X),(X),(X),(X),(X),(X),(X),49.4,(X)
4,1400000US13095000200,13095000200,"Census Tract 2, Dougherty County, Georgia",1006,141,643,118,200,90,363,...,(X),(X),(X),(X),(X),(X),(X),(X),47.4,(X)


In [15]:
new = df["GEO.display-label"].str.split(", ", n = 1, expand = True)   
df["tract"] = new[0] 
df["county"] = new[1] 

In [16]:
df = df.drop('GEO.display-label', axis = 1)
df.head(20)
df.shape

(44, 132)

In [17]:
l = len(df['tract'])
df2 = df.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df.loc[i, 'tract']
    m = re.search(re_string, s)
    df2.loc[i, 'tract'] = m.group(1)

In [18]:
df2['tract'] = df2['tract'].astype(str)

In [19]:
df2.shape
l = len(df2['tract'])
print(l)

44


In [20]:
tract_series = df2['tract']
l = len(df2['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = '000' + s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = '00' + s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = '0' + s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = '0' + df2['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = '00' + df2['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = '000' + df2['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df2['tract'][i] + '00'
df2.head()

,GEO.id,GEO.id2,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,...,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20,tract,county
0,Id,Id2,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,Nonfamily households; Margin of Error; Total,...,Households; Estimate; PERCENT ALLOCATED - Nonf...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Nonfam...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...,Geography,None
1,1400000US13007960100,13007960100,804,99,528,69,345,78,276,94,...,(X),(X),(X),(X),(X),(X),59.8,(X),960100,"Baker County, Georgia"
2,1400000US13007960200,13007960200,517,83,311,66,253,61,206,80,...,(X),(X),(X),(X),(X),(X),52.9,(X),960200,"Baker County, Georgia"
3,1400000US13095000100,13095000100,1905,220,911,173,244,117,994,210,...,(X),(X),(X),(X),(X),(X),49.4,(X),000100,"Dougherty County, Georgia"
4,1400000US13095000200,13095000200,1006,141,643,118,200,90,363,127,...,(X),(X),(X),(X),(X),(X),47.4,(X),000200,"Dougherty County, Georgia"


In [21]:
new1 = df2["county"].str.split(", ", n = 1, expand = True)   
df2["county"] = new1[0] 
df2["state"] = new1[1] 

In [30]:
for i in range(1,l):
    df2['HC01_EST_VC13'][i] = df2['HC01_EST_VC13'][i].astype(int)

AttributeError: 'str' object has no attribute 'astype'

In [38]:
df2.to_csv('edited_census_data.csv', index = False)

In [47]:
df2['HC01_EST_VC13'][1:] = df2['HC01_EST_VC13'][1:].astype(int)

In [44]:
df3 = df2.drop([0])
df3.index = range(df2.shape[0]-1)
type(df3['HC01_EST_VC13'][0])
df3.head()

,GEO.id,GEO.id2,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,...,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20,tract,county,state
0,1400000US13007960100,13007960100,804,99,528,69,345,78,276,94,...,(X),(X),(X),(X),(X),59.8,(X),960100,Baker County,Georgia
1,1400000US13007960200,13007960200,517,83,311,66,253,61,206,80,...,(X),(X),(X),(X),(X),52.9,(X),960200,Baker County,Georgia
2,1400000US13095000100,13095000100,1905,220,911,173,244,117,994,210,...,(X),(X),(X),(X),(X),49.4,(X),000100,Dougherty County,Georgia
3,1400000US13095000200,13095000200,1006,141,643,118,200,90,363,127,...,(X),(X),(X),(X),(X),47.4,(X),000200,Dougherty County,Georgia
4,1400000US13095000400,13095000400,2786,200,1446,202,623,191,1340,223,...,(X),(X),(X),(X),(X),34.6,(X),000400,Dougherty County,Georgia


In [43]:
df3.to_csv('newedited_census_data.csv', index = False)